In [2]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.utils.class_weight import compute_class_weight

print("🚀 Starting VGG16 model training...")

# Paths
train_dir = 'resized_dataset_split/train'
val_dir = 'resized_dataset_split/val'
img_size = (224, 224)
batch_size = 32
epochs = 25

# Data Generators
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    zoom_range=0.2,
    horizontal_flip=True,
    width_shift_range=0.1,
    height_shift_range=0.1
)
val_datagen = ImageDataGenerator(rescale=1./255)

train_data = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)

val_data = val_datagen.flow_from_directory(
    val_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

# Class weights
labels = train_data.classes
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(labels), y=labels)
class_weights = dict(enumerate(class_weights))
print("📊 Class weights:", class_weights)

# VGG16 model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Freeze base

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.3)(x)
predictions = Dense(train_data.num_classes, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Callbacks
callbacks = [
    EarlyStopping(patience=5, restore_best_weights=True),
    ReduceLROnPlateau(factor=0.2, patience=2)
]

# Training
history_vgg = model.fit(
    train_data,
    validation_data=val_data,
    epochs=epochs,
    class_weight=class_weights,
    callbacks=callbacks
)

# Save model
model.save("vgg16_balanced_model.h5")
print("✅ VGG16 training complete and saved as vgg16_balanced_model.h5")



🚀 Starting VGG16 model training...
Found 269 images belonging to 3 classes.
Found 69 images belonging to 3 classes.
📊 Class weights: {0: 1.6011904761904763, 1: 0.5977777777777777, 2: 1.4232804232804233}


c:\Users\venkatesh\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/25
9/9 ━━━━━━━━━━━━━━━━━━━━ 49s 5s/step - accuracy: 0.3758 - loss: 1.2956 - val_accuracy: 0.5797 - val_loss: 1.0601 - learning_rate: 0.0010
Epoch 2/25
9/9 ━━━━━━━━━━━━━━━━━━━━ 47s 5s/step - accuracy: 0.3814 - loss: 1.1944 - val_accuracy: 0.6377 - val_loss: 1.0329 - learning_rate: 0.0010
Epoch 3/25
9/9 ━━━━━━━━━━━━━━━━━━━━ 46s 5s/step - accuracy: 0.5194 - loss: 1.0513 - val_accuracy: 0.6667 - val_loss: 0.9879 - learning_rate: 0.0010
Epoch 4/25
9/9 ━━━━━━━━━━━━━━━━━━━━ 45s 5s/step - accuracy: 0.4185 - loss: 1.1189 - val_accuracy: 0.7391 - val_loss: 0.9499 - learning_rate: 0.0010
Epoch 5/25
9/9 ━━━━━━━━━━━━━━━━━━━━ 46s 5s/step - accuracy: 0.5118 - loss: 1.0088 - val_accuracy: 0.7681 - val_loss: 0.9130 - learning_rate: 0.0010
Epoch 6/25
9/9 ━━━━━━━━━━━━━━━━━━━━ 65s 8s/step - accuracy: 0.4412 - loss: 1.0906 - val_accuracy: 0.7826 - val_loss: 0.8946 - learning_rate: 0.0010
Epoch 7/25
9/9 ━━━━━━━━━━━━━━━━━━━━ 63s 7s/step - accuracy: 0.4722 - loss: 1.0046 - val_accuracy: 0.7826 - val_l

✅ VGG16 training complete and saved as vgg16_balanced_model.h5
